In [1]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

#data_raw = pd.read_csv('/home/sftpuser1/uploads/data.csv')
#data_raw

no docx
no docx


In [2]:
news_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?gid=0#gid=0', 
    1)
news_raw = news_raw[['link', 'news_date', 'summary',
       'summary_title']]
news_raw

,link,news_date,summary,summary_title
0,https://rb.ru/news/mistral-ai-invesrments/,2024-06-14 00:00:00,Французский стартап Mistral AI привлек €640 мл...,Главный европейский конкурент OpenAI привлек €...
1,https://36kr.com/p/2819048418101513?f=rss,2024-06-14 00:00:00,"В новость говорится о том, что 10 продуктов бы...",10 продуктов вошли в план «Фокус» на лето 2024...
2,https://36kr.com/newsflashes/2819214534330626?...,2024-06-14 00:00:00,Согласно сообщению на сайте Министерства комме...,Пресс-секретарь Министерства коммерции ответил...


In [3]:
import pandas as pd



df = news_raw.copy()

# Creating the text content
text_content = ""
for index, row in df.iterrows():
    text_content += f"Дата: {row['news_date']}\n"
    text_content += f"{row['summary_title']}\n"
    text_content += f"{row['summary']}\n"
    text_content += f"Ссылка: {row['link']}\n\n"

from pprint import pprint
pprint(text_content)

('Дата: 2024-06-14 00:00:00\n'
 'Главный европейский конкурент OpenAI привлек €640 млн от DST Global, General '
 'Catalyst и других\n'
 'Французский стартап Mistral AI привлек €640 млн при оценке €5,8 млрд. '
 'Инвестраунд серии B возглавили General Catalyst и DST Global. Это следует из '
 'данных Crunchbase, на которые обратил внимание RB.RU. источник: RB.RU\n'
 'Ссылка: https://rb.ru/news/mistral-ai-invesrments/\n'
 '\n'
 'Дата: 2024-06-14 00:00:00\n'
 '10 продуктов вошли в план «Фокус» на лето 2024 года\n'
 'В новость говорится о том, что 10 продуктов были выбраны для участия в '
 'программе «Фокус» на лето 2024 года. Среди них выделяется зарядное '
 'устройство Yoga от компании 图拉斯, которое прошло тестирование и получило '
 'положительные отзывы. Программа направлена на продвижение инновационных '
 'продуктов и технологий, что может повлиять на стратегию ИТ подразделений и '
 'металлургических компаний, заинтересованных в новых технологических '
 'решениях.\n'
 'Ссылка: https://36k

In [4]:
#!/usr/bin/env python
# coding: utf-8

import re
import time
import pandas as pd
from pprint import pprint
from gsheet_handler import GSheetHandler
import importlib
from google.oauth2 import service_account
from googleapiclient.discovery import build
import logging

module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Extract data from Google Sheets
news_raw = ggl.extract_data_from_google_sheet(
    'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?gid=0#gid=0', 1)
news_raw = news_raw[['link', 'news_date', 'summary', 'summary_title']]

# Copy dataframe
df = news_raw.copy()

# Create text content with markup for bold titles and clickable links
text_content = ""
requests = []
index = 1

for _, row in df.iterrows():
    row['news_date'] = pd.to_datetime(row['news_date']).strftime('%Y-%m-%d')
    
    start_index = index
    text_content += f"{row['summary_title']}\n"
    end_index = start_index + len(row['summary_title'])
    
    text_content += f"Дата: {row['news_date']}\n"
    text_content += f"{row['summary']}\n"
    link_start_index = index + len(f"{row['summary_title']}\n") + len(f"Дата: {row['news_date']}\n") + len(f"{row['summary']}\n")
    text_content += f"Ссылка: {row['link']}\n\n"
    link_end_index = link_start_index + len(f"Ссылка: {row['link']}")

    # Add requests for making summary_title bold
    requests.append({
        'updateTextStyle': {
            'range': {
                'startIndex': start_index,
                'endIndex': end_index
            },
            'textStyle': {
                'bold': True
            },
            'fields': 'bold'
        }
    })

    # Add requests for making other text not bold
    requests.append({
        'updateTextStyle': {
            'range': {
                'startIndex': end_index,
                'endIndex': link_start_index
            },
            'textStyle': {
                'bold': False
            },
            'fields': 'bold'
        }
    })

    # Add requests for making links clickable
    requests.append({
        'updateTextStyle': {
            'range': {
                'startIndex': link_start_index,
                'endIndex': link_end_index
            },
            'textStyle': {
                'link': {
                    'url': row['link']
                }
            },
            'fields': 'link'
        }
    })
    
    index += len(f"{row['summary_title']}\n") + len(f"Дата: {row['news_date']}\n") + len(f"{row['summary']}\n") + len(f"Ссылка: {row['link']}\n\n")

# Setup logging
logging.basicConfig(level=logging.DEBUG)

# Path to your service account file
SERVICE_ACCOUNT_FILE = 'google_sheet_key.json'
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service_docs = build('docs', 'v1', credentials=credentials)

# ID of your Google Docs document
DOCUMENT_ID = '1Rtfhw54kpkmuPmA7xpt9CVx5J_70kMiL636MQzl3dck'

# Function to update document with text
def update_document_with_text(document_id, text, requests):
    try:
        logging.info("Getting current document content")
        document = service_docs.documents().get(documentId=document_id).execute()
        
        logging.debug(f"Current document content: {document}")

        content = document.get('body').get('content')
        update_requests = []
        if len(content) > 1:
            end_index = document['body']['content'][-1]['endIndex'] - 1
            logging.debug(f"end_index: {end_index}")

            logging.info("Creating requests to delete content")
            update_requests.append({
                'deleteContentRange': {
                    'range': {
                        'startIndex': 1,
                        'endIndex': end_index
                    }
                }
            })

        logging.info("Creating requests to insert new text")
        update_requests.append({
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': text
            }
        })

        logging.info("Updating document")
        if update_requests:
            result = service_docs.documents().batchUpdate(documentId=document_id, body={'requests': update_requests}).execute()
            logging.info(f"Document updated with ID: {document_id}")
            
            # Apply styles
            result = service_docs.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
            logging.info(f"Styles applied to document ID: {document_id}")
        else:
            logging.info("No requests to update document")
    except Exception as e:
        logging.error(f"Error updating document: {e}")

# Update document with new text and apply styles
update_document_with_text(DOCUMENT_ID, text_content, requests)


no docx


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:root:Getting current document content
DEBUG:googleapiclient.discovery:URL being requested: GET https://docs.googleapis.com/v1/documents/1Rtfhw54kpkmuPmA7xpt9CVx5J_70kMiL636MQzl3dck?alt=json
DEBUG:google_auth_httplib2:Making request: POST https://oauth2.googleapis.com/token
DEBUG:root:Current document content: {'title': 'nlmk_template_didgest', 'body': {'content': [{'endIndex': 1, 'sectionBreak': {'sectionStyle': {'columnSeparatorStyle': 'NONE', 'contentDirection': 'LEFT_TO_RIGHT', 'sectionType': 'CONTINUOUS'}}}, {'startIndex': 1, 'endIndex': 96, 'paragraph': {'elements': [{'startIndex': 1, 'endIndex': 95, 'textRun': {'content': 'Главный европейский конкурент OpenAI привлек €640 млн от DST Global, General Catalyst и других', 'textStyle': {'bold': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.42745098, 'green': 0.39215687, 'blue': 0.9098039}}}, 'fontSize': {'magnitude': 12, 'unit': 